In [ ]:
from sqlalchemy import create_engine, text
import db_login
import pandas as pd
import numpy as np
import h3
import math

## Settings

In [ ]:
OSM_ID = 8269826
MAP_HEX_SIZE = 9
SELECTED_YEARS = (2023, 2024)
ARTIFICIAL_DATA_SCALING_FACTORS = "artificial_hex_scaling_factor.csv"
# List of H3 hex IDs or None
EXCLUDED_TARGET_HEXES = [
    h3.latlng_to_cell(50.2646, 18.975, MAP_HEX_SIZE),
]
COMMENT =  "artificial_S5P_scaled" # String or None
# List of dates for inference in the the "YYYY-mm-DD" format or None
SELECTED_DATES = [
    # "2023-02-26",
    # "2023-08-28",
    # "2023-10-15",
    # "2023-11-20",
    # "2024-01-21",
    # "2024-03-18",
    # "2024-09-16",
    # "2024-11-10",
]
SELECTED_INFERENCE_AREA = None # CSV with list of hexes or None
TEMPORAL_FRACTION = None # float or None
ARTIFICIAL_MIN_HEXES = [
    h3.latlng_to_cell(50.2444, 18.4376, MAP_HEX_SIZE),
    h3.latlng_to_cell(50.0394, 19.1668, MAP_HEX_SIZE),
]
DATASET_TYPE = "test" # one of "training", "test" or "inference"

In [ ]:
assert DATASET_TYPE in ["training", "test", "inference"], "Select correct dataset type"

## Database connection

In [ ]:
conn_template = "mysql+pymysql://{user}:{password}@{host}:{port}/{database}".format
conn = conn_template(
    user=db_login.user,
    password=db_login.passwd,
    host=db_login.host,
    port=int(db_login.port),
    database=db_login.db,
)
engine = create_engine(conn)

## Get meteo data

In [ ]:
query_meteo = """
SELECT 
    timestamp_utc,
    temperature,
    relative_humidity,
    pressure,
    dew_point,
    precipitation,
    wind_u,
    wind_v
FROM meteo.ERA5
WHERE YEAR(timestamp_utc) IN %(year)s;
"""

df_meteo = pd.read_sql_query(query_meteo, engine, params={"year": SELECTED_YEARS})
df_meteo.head()

In [ ]:
df_meteo["precipitation"] = np.where(
    df_meteo["precipitation"] >= df_meteo["precipitation"].shift(1),
    df_meteo["precipitation"] - df_meteo["precipitation"].shift(1),
    df_meteo["precipitation"],
)
df_meteo.loc[0, "precipitation"] = 0
df_meteo.head()

## Get traffic data

In [ ]:
query_traffic = """
SELECT timestamp_utc, daily_fraction, yearly_mean_fraction, mean_count
FROM traffic.SCPR
WHERE YEAR(timestamp_utc) IN %(year)s;
"""

df_traffic = pd.read_sql_query(query_traffic, engine, params={"year": SELECTED_YEARS})
df_traffic.rename(
    columns={
        "daily_fraction": "traffic_daily_fraction",
        "yearly_mean_fraction": "traffic_yearly_mean_fraction",\
        "mean_count": "traffic_mean_count",
    },
    inplace=True,
)
df_traffic.head()

## Get tree cover density

In [ ]:
query_tree_cover = """
SELECT tcd.h3_index, tcd.tree_cover_density
FROM clms.tree_cover_density tcd
INNER JOIN (
    SELECT h3_index
    FROM h3_maps.hex_%(hex_size)s
    WHERE osm_id = %(osm_id)s
) AS hex_map
ON tcd.h3_index=hex_map.h3_index;
"""

In [ ]:
df_tree_cover = pd.read_sql_query(query_tree_cover, engine, params={"hex_size": MAP_HEX_SIZE, "osm_id": OSM_ID})
df_tree_cover.head()

In [ ]:
df_tree_cover["tree_cover"] = (df_tree_cover["tree_cover_density"] / 100).round(3)
df_tree_cover.drop(columns=["tree_cover_density"], inplace=True)
df_tree_cover.head()

## Get grassland

In [ ]:
query_grassland = """
SELECT grs.h3_index, grs.grassland, grs.other
FROM clms.grassland grs
INNER JOIN (
    SELECT h3_index
    FROM h3_maps.hex_%(hex_size)s
    WHERE osm_id = %(osm_id)s
) AS hex_map
ON grs.h3_index=hex_map.h3_index;
"""

In [ ]:
df_grassland = pd.read_sql_query(query_grassland, engine, params={"hex_size": MAP_HEX_SIZE, "osm_id": OSM_ID})
df_grassland.head()

In [ ]:
cols_to_sum = df_grassland.columns.difference(["h3_index"])
df_grassland["sum"] = df_grassland[cols_to_sum].sum(axis=1)
df_grassland.head()

In [ ]:
cols_to_normalize = df_grassland.columns.difference(["h3_index", "sum"])
df_grassland[cols_to_normalize] = df_grassland[cols_to_normalize].div(df_grassland["sum"], axis=0)
df_grassland[cols_to_normalize] = df_grassland[cols_to_normalize].round(4)
df_grassland.drop(columns=["sum", "other"], inplace=True)
df_grassland.head()

## Get population

In [ ]:
query_population = """
SELECT pop.h3_index, pop.population
FROM ghsl.population pop
INNER JOIN (
    SELECT h3_index
    FROM h3_maps.hex_%(hex_size)s
    WHERE osm_id = %(osm_id)s
) AS hex_map
ON pop.h3_index=hex_map.h3_index;
"""

In [ ]:
df_population = pd.read_sql_query(query_population, engine, params={"hex_size": MAP_HEX_SIZE, "osm_id": OSM_ID})
df_population.head()

In [ ]:
def calc_pop_density(population, h3_index):
    pop_density = population / h3.cell_area(h3_index)

    return pop_density

In [ ]:
df_population["population_density"] = df_population.apply(lambda row: calc_pop_density(row.population, row.h3_index), axis=1).round(2)
df_population.drop(columns=["population"], inplace=True)
df_population.head()

## Get built-up characteristic

In [ ]:
query_builtup = """
SELECT
    bup.h3_index,
    bup.low_vegetation,
    bup.medium_vegetation,
    bup.high_vegetation,
    bup.water,
    bup.road,
    bup.residential_1,
    bup.residential_2,
    bup.residential_3,
    bup.residential_4,
    bup.residential_5,
    bup.`non-residential_1`,
    bup.`non-residential_2`,
    bup.`non-residential_3`,
    bup.`non-residential_4`,
    bup.`non-residential_5`,
    bup.other
FROM ghsl.builtup_c bup
INNER JOIN (
    SELECT h3_index
    FROM h3_maps.hex_%(hex_size)s
    WHERE osm_id = %(osm_id)s
) AS hex_map
ON bup.h3_index=hex_map.h3_index;
"""

In [ ]:
df_builtup = pd.read_sql_query(query_builtup, engine, params={"hex_size": MAP_HEX_SIZE, "osm_id": OSM_ID})
df_builtup.rename(columns={"other": "other_builtup"}, inplace=True)
df_builtup.head()

In [ ]:
cols_to_sum = df_builtup.columns.difference(["h3_index"])
df_builtup["sum"] = df_builtup[cols_to_sum].sum(axis=1)
df_builtup.head()

In [ ]:
cols_to_normalize = df_builtup.columns.difference(["h3_index", "sum"])
df_builtup[cols_to_normalize] = df_builtup[cols_to_normalize].div(df_builtup["sum"], axis=0)
df_builtup[cols_to_normalize] = df_builtup[cols_to_normalize].round(4)
df_builtup.drop(columns=["sum", "other_builtup"], inplace=True)
df_builtup.head()

## Get heat island data

In [ ]:
query_heat = """
SELECT hi.h3_index, hi.year, hi.temperature_mean AS temperature_anomaly
FROM meteo.heat_islands hi
INNER JOIN (
    SELECT h3_index
    FROM h3_maps.hex_%(hex_size)s
    WHERE osm_id = %(osm_id)s
) AS hex_map
ON hi.h3_index=hex_map.h3_index
WHERE hi.year IN %(year)s;
"""

In [ ]:
df_heat = pd.read_sql_query(query_heat, engine, params={"hex_size": MAP_HEX_SIZE, "osm_id": OSM_ID, "year": SELECTED_YEARS})
df_heat.head()

## Get NO2 concentrations from GIOŚ

In [ ]:
query_no2gios = """
SELECT stations.h3_index, gios.timestamp_utc, gios.value
FROM air_quality.NO2_GIOS gios
INNER JOIN (
	SELECT giosmeta.station_id, hex_map.h3_index
	FROM air_quality.gios_metadata giosmeta
	INNER JOIN (
	    SELECT h3_index
	    FROM h3_maps.hex_%(hex_size)s
	    WHERE osm_id = %(osm_id)s
	) AS hex_map
	ON giosmeta.hex_%(hex_size)s=hex_map.h3_index
) AS stations
ON gios.station_id=stations.station_id
WHERE YEAR(timestamp_utc) IN %(year)s;
"""

In [ ]:
df_no2gios = pd.read_sql_query(query_no2gios, engine, params={"hex_size": MAP_HEX_SIZE, "osm_id": OSM_ID, "year": SELECTED_YEARS})
df_no2gios.rename(columns={"value": "no2_gios"}, inplace=True)
df_no2gios.head()

In [ ]:
df_no2gios["timestamp_utc"] = pd.to_datetime(df_no2gios["timestamp_utc"])
df_no2gios["date"] = df_no2gios["timestamp_utc"].dt.date
df_no2gios["no2_gios"] = df_no2gios.groupby("date")["no2_gios"].transform(
    lambda row: row.fillna(row.mean())
)
df_no2gios.drop(columns=["date"], inplace=True)
df_no2gios.head()

In [ ]:
df_no2gios.dropna(inplace=True)

In [ ]:
if DATASET_TYPE == "training":
    df_no2gios = df_no2gios[~df_no2gios["h3_index"].isin(EXCLUDED_TARGET_HEXES)]
    
    df_scaling_f = pd.read_csv(ARTIFICIAL_DATA_SCALING_FACTORS)
    df_scaling_f.head()
    
    if len(ARTIFICIAL_MIN_HEXES) > 0:
        min_values = df_no2gios.groupby("timestamp_utc")["no2_gios"].min().reset_index()
        min_values.rename(columns={"no2_gios": "no2_gios_min"}, inplace=True)
        
        artificial_rows = []
        for hex_id in ARTIFICIAL_MIN_HEXES:
            df_temp = min_values.copy()
            df_temp["month"] = df_temp["timestamp_utc"].dt.month
            df_temp["h3_index"] = hex_id
            df_temp = pd.merge(df_temp, df_scaling_f, left_on=["h3_index", "month"], right_on=["h3_index", "month"], how="left")
            df_temp["no2_gios"] = df_temp["no2_gios_min"] * df_temp["scaling_factor"]
            artificial_rows.append(df_temp[["h3_index", "timestamp_utc", "no2_gios"]])
        
        artificial_data = pd.concat(artificial_rows, ignore_index=True)
        
        df_no2gios = pd.concat([df_no2gios, artificial_data], ignore_index=True)

In [ ]:
if DATASET_TYPE == "test":
    df_no2gios = df_no2gios[df_no2gios["h3_index"].isin(EXCLUDED_TARGET_HEXES)]

## Merge data

### Spatial data

In [ ]:
df_spatial = pd.merge(df_tree_cover, df_grassland, on="h3_index", how="outer")
df_spatial = pd.merge(df_spatial, df_population, on="h3_index", how="outer")
df_spatial = pd.merge(df_spatial, df_builtup, on="h3_index", how="outer")

df_spatial.fillna(0, inplace=True)
df_spatial.head()

### Temporal data

In [ ]:
df_temporal = pd.merge(df_meteo, df_traffic, on="timestamp_utc", how="outer")
df_temporal.fillna(0, inplace=True)
df_temporal.head()

### Create date related features

#### Encode day of year as sin and cos

In [ ]:
df_temporal["day_of_year"] = df_temporal["timestamp_utc"].dt.dayofyear
df_temporal["year_length"] = np.where(
    df_temporal["timestamp_utc"].dt.is_leap_year,
    366,
    365,
)

In [ ]:
def encode_doy(day_of_year, length_of_year):
    doy_sin = math.sin(2 * math.pi *  day_of_year / length_of_year)
    doy_cos = math.cos(2 * math.pi *  day_of_year / length_of_year)

    return doy_sin, doy_cos

In [ ]:
df_temporal[["day_of_year_sin", "day_of_year_cos"]] = df_temporal.apply(lambda row: encode_doy(row["day_of_year"], row["year_length"]), axis=1, result_type="expand")

#### Mark working days based on the day in week

In [ ]:
df_temporal["day_of_week"] = df_temporal["timestamp_utc"].dt.dayofweek
df_temporal["working_day"] = (df_temporal["day_of_week"] <= 4).astype(int)

#### Apply holidays to workings days

In [ ]:
df_holidays = pd.read_csv("holidays.csv")
df_holidays.head()

In [ ]:
df_temporal["not_holiday"] = (~df_temporal["timestamp_utc"].dt.strftime("%Y-%m-%d").isin(df_holidays["date"])).astype(int)
df_temporal["working_day"] = df_temporal["working_day"] * df_temporal["not_holiday"]

#### Remove temporary columns

In [ ]:
df_temporal.drop(columns=["day_of_year", "year_length", "day_of_week", "not_holiday"], inplace=True)

### Create dataset

In [ ]:
if DATASET_TYPE == "inference":
    available_dates = set(df_temporal["timestamp_utc"].dt.strftime("%Y-%m-%d"))

    if len(SELECTED_DATES) > 0:
        for date in SELECTED_DATES:
            assert date in available_dates, f"Data for {date} is not available"

In [ ]:
if  DATASET_TYPE in ["training", "test"]:
    df_output = pd.merge(df_no2gios, df_spatial, on="h3_index", how="left")
    df_output = pd.merge(df_output, df_temporal, on="timestamp_utc", how="left")
else:
    if len(SELECTED_DATES) > 0:
        df_temporal_selected = df_temporal[df_temporal["timestamp_utc"].dt.strftime("%Y-%m-%d").isin(SELECTED_DATES)]
        df_output = df_spatial.merge(df_temporal_selected, how="cross")
    else:
        if SELECTED_INFERENCE_AREA:
            df_inference_area = pd.read_csv(SELECTED_INFERENCE_AREA)
            df_spatial_selected = df_spatial[df_spatial["h3_index"].isin(df_inference_area[f"hex-{MAP_HEX_SIZE}"])]
            df_output = df_spatial_selected.merge(df_temporal, how="cross")
        else:
            df_output = df_spatial.merge(df_temporal, how="cross")

In [ ]:
df_output["year"] = df_output["timestamp_utc"].dt.year
df_output.head()

In [ ]:
df_output = pd.merge(df_output, df_heat, on=["h3_index", "year"], how="left")
df_output.drop(columns=["year"], inplace=True)
df_output.fillna(0, inplace=True)
df_output.head()

In [ ]:
df_output["temperature_trend_3h"] = df_output["temperature"] - df_output["temperature"].shift(3)
df_output["temperature_trend_3h"] = df_output["temperature_trend_3h"].round(1)
df_output["temperature_trend_6h"] = df_output["temperature"] - df_output["temperature"].shift(6)
df_output["temperature_trend_6h"] = df_output["temperature_trend_6h"].round(1)
df_output["relative_humidity_trend_3h"] = df_output["relative_humidity"] - df_output["relative_humidity"].shift(3)
df_output["relative_humidity_trend_3h"] = df_output["relative_humidity_trend_3h"].round(1)
df_output["relative_humidity_trend_6h"] = df_output["relative_humidity"] - df_output["relative_humidity"].shift(6)
df_output["relative_humidity_trend_6h"] = df_output["relative_humidity_trend_6h"].round(1)
df_output["pressure_trend_3h"] = df_output["pressure"] - df_output["pressure"].shift(3)
df_output["pressure_trend_3h"] = df_output["pressure_trend_3h"].round(1)
df_output["pressure_trend_6h"] = df_output["pressure"] - df_output["pressure"].shift(6)
df_output["pressure_trend_6h"] = df_output["pressure_trend_6h"].round(1)

df_output.fillna(0, inplace=True)
df_output.head()

### Test NO2 dataset

Feature parameters for all dates but only for selected test hex.

### Inference dataset

Feature parameters for all hexes in the area but limited to selected dates.

## Check for missing data

In [ ]:
df_output[df_output.isna().any(axis=1)]

## Export data to files

In [ ]:
years_str = "_".join(str(year) for year in SELECTED_YEARS)
base_filename = f"osm_{OSM_ID}_hex_{MAP_HEX_SIZE}_year_{years_str}.csv"
if COMMENT and DATASET_TYPE != "test":
    base_filename = base_filename.replace(".csv", f"_{COMMENT}.csv")

In [ ]:
df_output.to_csv(f"../data/NO2_{DATASET_TYPE}_dataset_{base_filename}", index=False)